#  COMP3210/6210: Assignment 2

## **Task 1:** Nearest Neighbor Search (refer to weeks 8 & 9 & 10 lecture notes)

###   Select ONE dataset (Restaurant, Shop, or Parking).

In [4]:
# Importing necessary library
import os

# Define the file path
file_path = os.path.expanduser('restaurant_dataset.txt')

# Function to read the dataset from a text file
def read_dataset(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            point_id = int(parts[0])
            x = float(parts[1])
            y = float(parts[2])
            data.append((point_id, x, y))
    return data

# Read the restaurant dataset
restaurant_data = read_dataset(file_path)

# Display the contents of the dataset
#for point in restaurant_data:
 #   print(point)


In [5]:
import os
import math
from rtree import index

# Function to read data from a text file
def read_dataset(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            point_id = int(parts[0])
            x = float(parts[1])
            y = float(parts[2])
            data.append((point_id, x, y))
    return data

# Function to compute Euclidean distance between two points
def euclidean_distance(point1, point2):
    return math.sqrt((point1[1] - point2[1]) ** 2 + (point1[2] - point2[2]) ** 2)

# Function to find the nearest neighbor for each query point in a dataset using Sequential Scan
def sequential_scan_nearest_neighbors(queries, dataset):
    nearest_neighbors = []
    for query in queries:
        min_distance = float('inf')
        nearest_point = None
        for point in dataset:
            distance = euclidean_distance(query, point)
            if distance < min_distance:
                min_distance = distance
                nearest_point = point
        nearest_neighbors.append((query[0], nearest_point[0], min_distance))
    return nearest_neighbors

# Function to create an R-tree index from a dataset
def create_rtree_index(dataset):
    p = index.Property()
    idx = index.Index(properties=p)
    for point in dataset:
        idx.insert(point[0], (point[1], point[2], point[1], point[2]), obj=point)
    return idx

# Function to find the nearest neighbor for each query point using the Best First algorithm with R-tree
def best_first_nearest_neighbors(queries, rtree_index):
    nearest_neighbors = []
    for query in queries:
        nearest = list(rtree_index.nearest((query[1], query[2], query[1], query[2]), 1, objects=True))[0].object
        distance = euclidean_distance(query, nearest)
        nearest_neighbors.append((query[0], nearest[0], distance))
    return nearest_neighbors

# Function to divide the dataset into two subspaces
def divide_dataset(dataset, dimension='x'):
    dataset_sorted = sorted(dataset, key=lambda point: point[1] if dimension == 'x' else point[2])
    mid = len(dataset_sorted) // 2
    return dataset_sorted[:mid], dataset_sorted[mid:]

# Function to find the nearest neighbor for each query point using BF with Divide-and-Conquer
def bf_divide_and_conquer_nearest_neighbors(queries, dataset):
    subspace1, subspace2 = divide_dataset(dataset)
    rtree_index1 = create_rtree_index(subspace1)
    rtree_index2 = create_rtree_index(subspace2)
    
    nearest_neighbors = []
    for query in queries:
        nearest1 = list(rtree_index1.nearest((query[1], query[2], query[1], query[2]), 1, objects=True))[0].object
        nearest2 = list(rtree_index2.nearest((query[1], query[2], query[1], query[2]), 1, objects=True))[0].object
        distance1 = euclidean_distance(query, nearest1)
        distance2 = euclidean_distance(query, nearest2)
        if distance1 < distance2:
            nearest_neighbors.append((query[0], nearest1[0], distance1))
        else:
            nearest_neighbors.append((query[0], nearest2[0], distance2))
    return nearest_neighbors

# Paths to dataset files
restaurant_file = os.path.expanduser('restaurant_dataset.txt')
queries_file = os.path.expanduser('query_points.txt')

# Read datasets and queries
restaurant_data = read_dataset(restaurant_file)
user_queries = read_dataset(queries_file)

# Sequential Scan
nearest_restaurants_seq = sequential_scan_nearest_neighbors(user_queries, restaurant_data)
print("Nearest Restaurants using Sequential Scan:")
for result in nearest_restaurants_seq:
    print(f"User {result[0]} -> Restaurant {result[1]} (Distance: {result[2]:.2f})")

# Best First with R-tree
restaurant_rtree_index = create_rtree_index(restaurant_data)
nearest_restaurants_bf = best_first_nearest_neighbors(user_queries, restaurant_rtree_index)
print("\nNearest Restaurants using Best First Algorithm with R-tree:")
for result in nearest_restaurants_bf:
    print(f"User {result[0]} -> Restaurant {result[1]} (Distance: {result[2]:.2f})")

# BF with Divide-and-Conquer
nearest_restaurants_bf_dc = bf_divide_and_conquer_nearest_neighbors(user_queries, restaurant_data)
print("\nNearest Restaurants using BF with Divide-and-Conquer:")
for result in nearest_restaurants_bf_dc:
    print(f"User {result[0]} -> Restaurant {result[1]} (Distance: {result[2]:.2f})")


Nearest Restaurants using Sequential Scan:
User 1 -> Restaurant 121862 (Distance: 0.12)
User 2 -> Restaurant 12827 (Distance: 0.10)
User 3 -> Restaurant 113975 (Distance: 0.08)
User 4 -> Restaurant 149159 (Distance: 0.12)
User 5 -> Restaurant 94300 (Distance: 0.09)
User 6 -> Restaurant 43818 (Distance: 0.15)
User 7 -> Restaurant 8816 (Distance: 0.11)
User 8 -> Restaurant 79795 (Distance: 0.07)
User 9 -> Restaurant 4586 (Distance: 0.14)
User 10 -> Restaurant 67804 (Distance: 0.18)
User 11 -> Restaurant 11425 (Distance: 0.22)
User 12 -> Restaurant 85208 (Distance: 0.04)
User 13 -> Restaurant 100197 (Distance: 0.13)
User 14 -> Restaurant 58292 (Distance: 0.07)
User 15 -> Restaurant 64709 (Distance: 0.05)
User 16 -> Restaurant 97516 (Distance: 0.04)
User 17 -> Restaurant 74801 (Distance: 0.09)
User 18 -> Restaurant 58974 (Distance: 0.09)
User 19 -> Restaurant 93541 (Distance: 0.07)
User 20 -> Restaurant 34450 (Distance: 0.08)
User 21 -> Restaurant 89802 (Distance: 0.12)
User 22 -> Restaura

##  **Output:**  For each algorithm (Sequential Scan Based, BF Algorithm, and BF with Divideand-Conquer), display and output the following information in a single txt file:  The ID, x, and y coordinates of the nearest neighbor identified for each query point

In [6]:
import os
import math
from time import time
from rtree import index

# Function to read data from a text file
def read_dataset(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            point_id = int(parts[0])
            x = float(parts[1])
            y = float(parts[2])
            data.append((point_id, x, y))
    return data

# Function to compute Euclidean distance between two points
def euclidean_distance(point1, point2):
    return math.sqrt((point1[1] - point2[1]) ** 2 + (point1[2] - point2[2]) ** 2)

# Function to find the nearest neighbor for each query point in a dataset using Sequential Scan
def sequential_scan_nearest_neighbors(queries, dataset):
    nearest_neighbors = []
    for query in queries:
        min_distance = float('inf')
        nearest_point = None
        for point in dataset:
            distance = euclidean_distance(query, point)
            if distance < min_distance:
                min_distance = distance
                nearest_point = point
        nearest_neighbors.append((query[0], nearest_point[0], nearest_point[1], nearest_point[2], min_distance))
    return nearest_neighbors

# Function to create an R-tree index from a dataset
def create_rtree_index(dataset):
    p = index.Property()
    idx = index.Index(properties=p)
    for point in dataset:
        idx.insert(point[0], (point[1], point[2], point[1], point[2]), obj=point)
    return idx

# Function to find the nearest neighbor for each query point using the Best First algorithm with R-tree
def best_first_nearest_neighbors(queries, rtree_index):
    nearest_neighbors = []
    for query in queries:
        nearest = list(rtree_index.nearest((query[1], query[2], query[1], query[2]), 1, objects=True))[0].object
        distance = euclidean_distance(query, nearest)
        nearest_neighbors.append((query[0], nearest[0], nearest[1], nearest[2], distance))
    return nearest_neighbors

# Function to divide the dataset into two subspaces
def divide_dataset(dataset, dimension='x'):
    dataset_sorted = sorted(dataset, key=lambda point: point[1] if dimension == 'x' else point[2])
    mid = len(dataset_sorted) // 2
    return dataset_sorted[:mid], dataset_sorted[mid:]

# Function to find the nearest neighbor for each query point using BF with Divide-and-Conquer
def bf_divide_and_conquer_nearest_neighbors(queries, dataset):
    subspace1, subspace2 = divide_dataset(dataset)
    rtree_index1 = create_rtree_index(subspace1)
    rtree_index2 = create_rtree_index(subspace2)
    
    nearest_neighbors = []
    for query in queries:
        nearest1 = list(rtree_index1.nearest((query[1], query[2], query[1], query[2]), 1, objects=True))[0].object
        nearest2 = list(rtree_index2.nearest((query[1], query[2], query[1], query[2]), 1, objects=True))[0].object
        distance1 = euclidean_distance(query, nearest1)
        distance2 = euclidean_distance(query, nearest2)
        if distance1 < distance2:
            nearest_neighbors.append((query[0], nearest1[0], nearest1[1], nearest1[2], distance1))
        else:
            nearest_neighbors.append((query[0], nearest2[0], nearest2[1], nearest2[2], distance2))
    return nearest_neighbors

# Paths to dataset files
restaurant_file = os.path.expanduser('restaurant_dataset.txt')
queries_file = os.path.expanduser('query_points.txt')

# Read datasets and queries
restaurant_data = read_dataset(restaurant_file)
user_queries = read_dataset(queries_file)

# Sequential Scan
start_time = time()
nearest_restaurants_seq = sequential_scan_nearest_neighbors(user_queries, restaurant_data)
total_time_seq = time() - start_time
average_time_seq = total_time_seq / len(user_queries)

# Best First with R-tree
restaurant_rtree_index = create_rtree_index(restaurant_data)
start_time = time()
nearest_restaurants_bf = best_first_nearest_neighbors(user_queries, restaurant_rtree_index)
total_time_bf = time() - start_time
average_time_bf = total_time_bf / len(user_queries)

# BF with Divide-and-Conquer
start_time = time()
nearest_restaurants_bf_dc = bf_divide_and_conquer_nearest_neighbors(user_queries, restaurant_data)
total_time_bf_dc = time() - start_time
average_time_bf_dc = total_time_bf_dc / len(user_queries)

# Write results to a text file
output_file = os.path.expanduser('nearest_neighbors_results.txt')

with open(output_file, 'w') as file:
    file.write("Nearest Restaurants using Sequential Scan:\n")
    for result in nearest_restaurants_seq:
        file.write(f"Query {result[0]} -> id={result[1]}, x={result[2]}, y={result[3]} (Distance: {result[4]:.2f})\n")
    file.write(f"Total running time: {total_time_seq:.2f} seconds\n")
    file.write(f"Average time per query: {average_time_seq:.4f} seconds\n\n")
    
    file.write("Nearest Restaurants using Best First Algorithm with R-tree:\n")
    for result in nearest_restaurants_bf:
        file.write(f"Query {result[0]} -> id={result[1]}, x={result[2]}, y={result[3]} (Distance: {result[4]:.2f})\n")
    file.write(f"Total running time: {total_time_bf:.2f} seconds\n")
    file.write(f"Average time per query: {average_time_bf:.4f} seconds\n\n")
    
    file.write("Nearest Restaurants using BF with Divide-and-Conquer:\n")
    for result in nearest_restaurants_bf_dc:
        file.write(f"Query {result[0]} -> id={result[1]}, x={result[2]}, y={result[3]} (Distance: {result[4]:.2f})\n")
    file.write(f"Total running time: {total_time_bf_dc:.2f} seconds\n")
    file.write(f"Average time per query: {average_time_bf_dc:.4f} seconds\n")

print("Results have been written to nearest_neighbors_results.txt")


Results have been written to nearest_neighbors_results.txt


#  Task 2: Skyline Search (refer to weeks 8 & 9 & 10 lecture notes)



In [7]:
import os
from rtree import index
import heapq
import time

# Function to read data from a text file
def read_dataset(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            point_id = int(parts[0])
            cost = float(parts[1])
            size = float(parts[2])
            data.append((point_id, cost, size))
    return data

# Function to create an R-tree index from the dataset
def create_rtree_index(dataset):
    p = index.Property()
    idx = index.Index(properties=p)
    for point in dataset:
        idx.insert(point[0], (point[1], point[2], point[1], point[2]), obj=point)
    return idx

# 1. Sequential Scan Based Method
def sequential_scan_skyline(dataset):
    start_time = time.time()
    skyline = []
    for p in dataset:
        if not any(p[1] <= q[1] and p[2] <= q[2] for q in dataset if p != q):
            skyline.append(p)
    end_time = time.time()
    execution_time = end_time - start_time
    return skyline, execution_time

# 2. Branch and Bound Skyline (BBS) Algorithm
def bbs_skyline(dataset):
    start_time = time.time()
    rtree_index = create_rtree_index(dataset)
    skyline = []
    heap = []
    dominated = set()

    for point in dataset:
        if not any(point[1] <= p[1] and point[2] <= p[2] for p in dataset if p != point):
            heapq.heappush(heap, point)
            dominated.add(point[0])

    while heap:
        p = heapq.heappop(heap)
        skyline.append(p)
        for q_id in rtree_index.intersection((p[1], p[2], p[1], p[2]), objects=True):
            q = next((obj for obj in dataset if obj[0] == q_id), None)
            if q and q[0] not in dominated and q[1] <= p[1] and q[2] <= p[2]:
                dominated.add(q[0])
                heapq.heappush(heap, q)

    end_time = time.time()
    execution_time = end_time - start_time
    return skyline, execution_time

# 3. BBS with Divide-and-Conquer
def divide_dataset(dataset, dimension='x'):
    dataset_sorted = sorted(dataset, key=lambda point: point[1] if dimension == 'x' else point[2])
    mid = len(dataset_sorted) // 2
    return dataset_sorted[:mid], dataset_sorted[mid:]

def bbs_divide_and_conquer_skyline(dataset):
    def one_d_dominance(points):
        skyline = []
        min_size = float('inf')
        for point in points:
            if point[2] < min_size:
                skyline.append(point)
                min_size = point[2]
        return skyline

    if len(dataset) <= 1:
        return dataset, 0.0

    start_time = time.time()

    left, right = divide_dataset(dataset, 'cost')  # Divide by cost (X dimension)

    left_skyline, _ = bbs_divide_and_conquer_skyline(left)
    right_skyline, _ = bbs_divide_and_conquer_skyline(right)

    combined_skyline = left_skyline + right_skyline
    combined_skyline.sort(key=lambda point: point[2])  # Sort by size (Y dimension)

    skyline = one_d_dominance(combined_skyline)

    end_time = time.time()
    execution_time = end_time - start_time

    return skyline, execution_time

# Paths to dataset files
dataset_directory = os.path.expanduser("")
selected_dataset = 'city1'  # Replace with 'city1', 'city2', or 'city3'
dataset_file = os.path.join(dataset_directory, f'{selected_dataset}.txt')

# Read the selected dataset
dataset = read_dataset(dataset_file)

# Perform Sequential Scan Based Method
skyline_seq, seq_time = sequential_scan_skyline(dataset)
print(f"Sequential Scan Execution Time: {seq_time:.6f} seconds")

# Perform Branch and Bound Skyline calculation
skyline_bbs, bbs_time = bbs_skyline(dataset)
print(f"Branch and Bound Execution Time: {bbs_time:.6f} seconds")

# Perform BBS with Divide-and-Conquer Skyline calculation
skyline_bbs_dc, bbs_dc_time = bbs_divide_and_conquer_skyline(dataset)
print(f"BBS with Divide-and-Conquer Execution Time: {bbs_dc_time:.6f} seconds")


Sequential Scan Execution Time: 3.763429 seconds
Branch and Bound Execution Time: 48.338260 seconds
BBS with Divide-and-Conquer Execution Time: 2.409142 seconds


# Task 3: Project Report (refer to the uploaded template)

# - A breakdown of the tasks and responsibilities allocated to each member of the group.
# - An in-depth description of the primary functions within your source code (Tasks 1and 2).

### Task 1: Nearest Neighbor Search

#### Models Used:

1. **Sequential Scan Based Method**:
   - **Concept**: This method involves a straightforward approach where each query point is compared against every point in the dataset to find the nearest neighbor.
   - **Operation**: For each query point, calculate the distance to every point in the dataset and identify the point with the minimum distance.
   - **Algorithm Efficiency**: Simple to implement but computationally expensive, especially for large datasets, due to its O(n) complexity for each query.

2. **Best First (BF) Algorithm with R-Tree**:
   - **Concept**: Utilizes an R-Tree data structure to improve efficiency by organizing spatial data for faster spatial queries.
   - **Operation**: The R-Tree organizes points into a hierarchy of nodes, where each node's bounding box (MBR) encompasses its child nodes or data points.
   - **Algorithm Efficiency**: Reduces the number of distance calculations by only considering points within relevant nodes of the R-Tree. This makes it more efficient than sequential scanning, particularly for high-dimensional and large datasets.

3. **BF with Divide-and-Conquer**:
   - **Concept**: Extends the BF algorithm by partitioning the dataset into smaller subspaces (using dimensions like X or Y) and applying the BF algorithm recursively.
   - **Operation**: Divides the dataset based on one dimension, constructs an R-Tree for each subspace, and applies the BF algorithm to find the nearest neighbors in each partition.
   - **Algorithm Efficiency**: Balances the search complexity by reducing the search space in each recursion step, thereby improving efficiency compared to a single R-Tree approach.

#### Explanation:

- **Sequential Scan**: 
  - Suitable for small datasets or initial prototype development due to its simplicity.
  - Inefficient for large datasets because it checks every point.

- **BF Algorithm with R-Tree**:
  - Efficient for high-dimensional data and large datasets.
  - Uses spatial indexing (R-Tree) to prune irrelevant search spaces, reducing computation time significantly.

- **BF with Divide-and-Conquer**:
  - Balances the load by dividing the dataset into manageable subspaces.
  - Combines spatial indexing with recursive partitioning, enhancing efficiency for complex spatial queries.

### Task 2: Skyline Search

#### Models Used:

1. **Sequential Scan Based Method**:
   - **Concept**: Identifies skyline points by sequentially evaluating whether each data point is dominated by any other points in the dataset.
   - **Operation**: Compares each data point against all others to determine if it should be included in the skyline (not dominated by any other point).
   - **Algorithm Efficiency**: Simple to implement but computationally expensive, especially for large datasets, due to its O(n^2) complexity.

2. **Branch and Bound Skyline (BBS) Algorithm with R-Tree**:
   - **Concept**: Utilizes an R-Tree to partition the dataset and efficiently prune dominated points.
   - **Operation**: Constructs an R-Tree with MBRs for each data point, then uses a priority queue to iteratively find skyline points.
   - **Algorithm Efficiency**: Reduces computational overhead by leveraging spatial indexing (R-Tree) for dominance checking, making it suitable for large-scale skyline queries.

3. **BBS with Divide-and-Conquer**:
   - **Concept**: Enhances BBS by recursively dividing the dataset and applying the BBS algorithm on each partition.
   - **Operation**: Divides the dataset into smaller subspaces (e.g., based on cost or size), constructs R-Trees for each partition, and combines skyline results using dominance screening.
   - **Algorithm Efficiency**: Optimizes the skyline search by balancing workload across partitions and leveraging spatial indexing, thus improving overall efficiency.

#### Explanation:

- **Sequential Scan**:
  - Straightforward but inefficient for large datasets due to quadratic complexity.
  - Suitable for small datasets where efficiency is less critical.

- **Branch and Bound Skyline (BBS) with R-Tree**:
  - Efficiently handles dominance checks using spatial indexing (R-Tree).
  - Reduces computational complexity by focusing on relevant data partitions.

- **BBS with Divide-and-Conquer**:
  - Optimizes skyline search by recursively partitioning and combining results.
  - Balances computational load and improves efficiency compared to a single-tree approach.

#  A clear specification of the requirements for running your code, including the required operating system environment, the location of input files, and any essential input parameters, etc

Install rtree and ensure all files are in the same folder as the code file.

